In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
import sys
sys.path.append('/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark')

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import glob
import os
import os.path as op
import random
import time
import warnings
warnings.filterwarnings("ignore")

import h5py
from joblib import Parallel, delayed
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

from mri.numerics.gradient import GradAnalysis2

from cascading import cascade_net
from data import ZeroFilled2DSequence, MaskShifted2DSequence, MaskedUntouched2DSequence
from evaluate import METRIC_FUNCS, Metrics
from fourier import FFT2
from utils import gen_mask, keras_psnr, keras_ssim
from reconstruction import reco_unet_from_test_file, reco_and_gt_unet_from_val_file, reco_and_gt_unet_from_val_file_no_norm, reco_and_gt_net_from_val_file

Using TensorFlow backend.



                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=0.18.0  - required | 1.3.0     installed
numpy          : >=1.11.0  - required | 1.16.4    installed
matplotlib     : >=2.0.0   - required | 3.1.0     installed
future         : >=0.16.0  - required | 0.17.1    installed
astropy        : ==2.0.8   - required | 2.0.8     installed
nibabel        : >=2.1.0   - required | 2.4.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.1.5   - requi

In [4]:
np.random.seed(0)

In [5]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [6]:
fourier_op = FFT2(np.array([1]))

In [7]:
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
val_reco_path = '/media/Zaccharie/UHRes/singlecoil_val_reco/'
val_filenames = glob.glob(val_path + '*')

In [8]:
AF = 4

In [9]:
val_gen = MaskedUntouched2DSequence(val_path, mode='validation', af=AF)

In [10]:
def metrics_for_args(model):
    metrics = Metrics(METRIC_FUNCS)
    pred_and_gt = [
#         reco_and_gt_unet_from_val_file(*val_gen[i], model) 
        reco_and_gt_net_from_val_file(*val_gen[i], model)
#         reco_and_gt_unet_from_val_file_no_norm(*val_gen[i], model) 
        for i in tqdm_notebook(range(len(val_gen)), desc='Val files')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Stats'):
        metrics.push(images, im_recos)
    return metrics

In [11]:
# run_id = f'unet_wo_lastrelu_af{AF}_test'
run_id = f'cascadenet_af{AF}_1566919402'
epoch = 200
chkpt_path = f'checkpoints/{run_id}-{epoch}.hdf5'
# unet = load_model(chkpt_path, custom_objects={'keras_psnr': keras_psnr,'keras_ssim': keras_ssim})
run_params = {
    'n_cascade': 5,
    'n_convs': 5,
    'n_filters': 64,
    'noiseless': True,
}
cascade_net = cascade_net(**run_params)
cascade_net.load_weights(chkpt_path)

W0828 13:51:10.843884 140194992424704 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 13:51:10.869534 140194992424704 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:63: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0828 13:51:10.903230 140194992424704 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 13:51:10.911182 140194992424704 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.ra

In [12]:
metrics = metrics_for_args(cascade_net)

In [13]:
metrics

PSNR = 31.94 +/- 6.999 SSIM = 0.7185 +/- 0.2721